# 1. NLG

Seção para realizar a prepração do ambiente de desenvolvimento, realizando a conexão com o Google Drive e importando as bibliotecas necessárias e importação das bases fornecidas, transformando todas em uma só.


In [1]:
!pip install llama-recipes ipywidgets
!pip uninstall -qqq pyarrow
!pip install -qqq pyarrow==15.0.2 --progress-bar off
!pip install datasets --progress-bar off
!pip install -qqq accelerate==0.31.0 --progress-bar off
!pip install -qqq bitsandbytes==0.43.1 --progress-bar off
!pip install -qqq peft==0.11.1 --progress-bar off
!pip install -qqq trl --progress-bar off
!pip install -qqq colored==2.2.4 --progress-bar off
!pip install -qqq sentence-transformers --progress-bar off

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
INFO: pip is looking at multiple versions of openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of optimum to determine which version

Proceed (Y/n)? Y
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 15.0.2 which is incompatible.


In [2]:
# Importar bibliotecas necessárias
import pandas as pd
import re

In [3]:
## Carregamento dos dados
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Alterar caminhos para onde estão armazenadas a bases
PATH = '/content/drive/MyDrive/DataSetM11/base1_v1_20240904.csv'
PATH2 = '/content/drive/MyDrive/DataSetM11/base2_v1_20240904.xlsx'
PATH3 = '/content/drive/MyDrive/DataSetM11/base3_v1_20240904.xlsx'

base = pd.read_csv(PATH, sep=';')
base2 = pd.read_excel(PATH2, sheet_name='Q&A')
base3 = pd.read_excel(PATH3)

### Tratamento básico das bases

Retirar linhas duplicadas, filtrar colunas principais e exploração da base.

In [5]:
# Filtrando apenas colunas importantes retirando os Index
base = base[['Intencao','Pergunta','Resposta\n']]
base2 = base2[['Intencao', 'Pergunta','Resposta\n']]
base3 = base3[['Intencao', 'Pergunta','Resposta\n']]

In [6]:
print('Tamanho database 1: ', len(base))
print('Tamanho database 2: ', len(base2))
print('Tamanho database 3: ', len(base3))

Tamanho database 1:  981
Tamanho database 2:  999
Tamanho database 3:  728


In [7]:
# Junção das bases de dados
baseGeral = pd.merge(base, base2, how='outer')
baseGeral = pd.merge(baseGeral, base3, how='outer')

In [8]:
# Renomear colunas
baseGeral.rename(columns={'Resposta\n':'Resposta'}, inplace=True)

In [9]:
# Retirar linhas que estão duplicadas
baseGeral.drop_duplicates(inplace=True)

In [10]:
# Visualização
baseGeral.head()

,Intencao,Pergunta,Resposta
0,Como depositar,Boa dia.tudo bem?eu gostaria de saber sobre aq...,"Bom dia! Sim, o sr pode utilizar o cartão de d..."
1,Como fazer remessa,Como enviar dinheiro do Japão?,"Para se inscrever no serviço de remessa, por f..."
2,Tempo de remessa,Quanto tempo levará para o beneficiário recebe...,"Via de regra, as remessas serão pagas via PIX ..."
152,"Pedido de envio via metodo ""ByPhone""",Boa tarde\nAcabei de fazer a transferência de ...,iremos processar a sua solicitacao.\nMuito obr...
153,"Pedido de envio via metodo ""ByPhone""",Poderia fazer a remessa de 22yenes para o BBB ...,iremos processar a sua solicitacao.\nMuito obr...


In [11]:
len(baseGeral)

799

# 2. Pipeline

**Etapa importante para lidar com os textos do dataframe futuramente.**

**Nesta etapa, o processo de pré-processamento de dados foi separado nas seguintes etapas a seguir:**

Limpeza dos dados do dataframe: remover os espaços em excesso, converter o texto em uma frase de uma linha, normalizar o texto para que fique todo com letras minúsculas, remover as pontuações, números, símbolos, caracteres especiais e emojis, links e endereços de e-mail;

## Normalização e remoção de dados que não serão utilizados

In [12]:
# Dar espaços após as pontuações
baseGeral['Pergunta'] = baseGeral['Pergunta'].apply(lambda x: re.sub(r'([.,!?;:])', r'\1 ', str(x)))
baseGeral['Resposta'] = baseGeral['Resposta'].apply(lambda x: re.sub(r'([.,!?;:])', r'\1 ', str(x)))

# Substituir quebras de linha por espaços
baseGeral['Pergunta'] = baseGeral['Pergunta'].apply(lambda x: re.sub(r'\n', ' ', str(x)))
baseGeral['Resposta'] = baseGeral['Resposta'].apply(lambda x: re.sub(r'\n', ' ', str(x)))

# Normalizar o dataframe para letras minúsculas
baseGeral['Pergunta'] = baseGeral['Pergunta'].str.lower()
baseGeral['Resposta'] = baseGeral['Resposta'].str.lower()

# Remover pontuações do dataframe
baseGeral['Pergunta'] = baseGeral['Pergunta'].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))
baseGeral['Resposta'] = baseGeral['Resposta'].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))

# Remover números do dataframe
baseGeral['Pergunta'] = baseGeral['Pergunta'].apply(lambda x: re.sub(r'\d+', '', str(x)))
baseGeral['Resposta'] = baseGeral['Resposta'].apply(lambda x: re.sub(r'\d+', '', str(x)))

# Remover espaços extras do dataframe
baseGeral['Pergunta'] = baseGeral['Pergunta'].apply(lambda x: re.sub(r'\s+', ' ', str(x)))
baseGeral['Resposta'] = baseGeral['Resposta'].apply(lambda x: re.sub(r'\s+', ' ', str(x)))

# Remover URLs
baseGeral['Pergunta'] = baseGeral['Pergunta'].apply(lambda x: re.sub(r'http\S+', '', str(x)))
baseGeral['Resposta'] = baseGeral['Resposta'].apply(lambda x: re.sub(r'http\S+', '', str(x)))

# Remover emails
baseGeral['Pergunta'] = baseGeral['Pergunta'].apply(lambda x: re.sub(r'\S+@\S+', '', str(x)))
baseGeral['Resposta'] = baseGeral['Resposta'].apply(lambda x: re.sub(r'\S+@\S+', '', str(x)))

# Remover emojis
baseGeral['Pergunta'] = baseGeral['Pergunta'].apply(lambda x: re.sub(r'[^\x00-\x7F]+', '', str(x)))
baseGeral['Resposta'] = baseGeral['Resposta'].apply(lambda x: re.sub(r'[^\x00-\x7F]+', '', str(x)))

# Remover palavras que contém apenas um caractere como aaa, bbbb
baseGeral['Pergunta'] = baseGeral['Pergunta'].apply(lambda x: re.sub(r'\b(\w)\1+\b', '', str(x)))
baseGeral['Resposta'] = baseGeral['Resposta'].apply(lambda x: re.sub(r'\b(\w)\1+\b', '', str(x)))


## Dicionário das abreviações encontradas na base de dados

Dicionário para conversão de abreviações para palavras por extenso: foi feita uma análise das principais abreviações que apareciam no dataframe e, com base nisso, criado um dicionário que é usado para substituir cada abreviação por sua palavra por extenso através de um loop que percorre as principais colunas que estamos usando;


In [13]:
# Criação de um dicionário para tratar abreviações do dataframe
abreviacoes = {
    'add': 'adicionar',
    'app': 'aplicativo',
    'pq': 'porque',
    'vc': 'você',
    'vcs': 'vocês',
    'sr': 'senhor',
    'sra': 'senhora',
    'srto': 'senhorito',
    'srta': 'senhorita',
    'qdo': 'quando',
    'qdo': 'quando',
    'qt': 'quanto',
    'qto': 'quanto',
    'pra': 'para',
    'q': 'que',
    'p': 'para',
    'pf': 'por favor',
    'yenes': 'ienes',
    'yens': 'ienes',
    'iens': 'ienes',
    'hj': 'hoje',
    'obg': 'obrigado',
    'td': 'tudo',
    'bm': 'bom',
}

In [14]:
# Substituindo as abreviações das palavras para extenso em toda a base de dados de uma vez
def substituir_abreviacoes(texto, abreviacoes):
    for abreviacao, palavra in abreviacoes.items():
        texto = texto.str.replace(r'\b' + abreviacao + r'\b', palavra, regex=True)
    return texto

# Aplicando a função para substituir abreviações nas colunas específicas
baseGeral['Pergunta'] = substituir_abreviacoes(baseGeral['Pergunta'], abreviacoes)
baseGeral['Resposta'] = substituir_abreviacoes(baseGeral['Resposta'], abreviacoes)

In [15]:
baseGeral.to_excel('/content/drive/MyDrive/DataSetM11/baseTratadaNLG.xlsx')

## Baixar base tratada

In [16]:
import random
from textwrap import dedent
from typing import Dict, List

import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from colored import Back, Fore, Style
from datasets import Dataset, load_dataset
from matplotlib.ticker import PercentFormatter
from peft import (
    LoraConfig,
    PeftModel,
    TaskType,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
)
from trl import DataCollatorForCompletionOnlyLM, SFTConfig, SFTTrainer

In [17]:
# uncomment if running from Colab T4

import huggingface_hub
huggingface_hub.login()

In [29]:
from transformers import LlamaForCausalLM, AutoTokenizer
MODEL_NAME = "meta-llama/Meta-Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    use_fast=True
)
tokenizer.add_special_tokens({'pad_token': "<|pad|>"})
tokenizer.padding_side = "right"



In [19]:
from textwrap import dedent
def format_example(row):
    prompt = dedent(
        f"""
        Intenção: {row['Intencao']}
        Responda a nova mensagem do usuário: {row['Pergunta']}
        """
    )
    messages = [
        {"role": "system",
         "content": """ Você é uma IA generativa que atua como atendente de remessas da Brastel Remit.
        Seu trabalho é responder as dúvidas dos clientes e solucionar problemas relacionados a remessas internacionais.
        Se for solicitar informações do usuário, faça quebra de linha mais de uma vez para facilitar a leitura e adicione emojis para representar o que foi solicitado.
        Leve em consideração a Intenção da pergunta do usuário para formular a resposta.""",
        },
        {"role": "user",
         "content": prompt,
        },
        {"role": "assistant",
         "content": row['Resposta'],
        },
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False)

### Sem Data Augmentation

In [20]:
baseGeral["prompt"] = baseGeral.apply(format_example, axis=1)

## Llama-3.1-8B

In [ ]:

import torch
from llama_recipes.configs import train_config as TRAIN_CONFIG

train_config = TRAIN_CONFIG()
train_config.model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
train_config.num_epochs = 1
train_config.run_validation = False
train_config.gradient_accumulation_steps = 4
train_config.batch_size_training = 1
train_config.lr = 3e-4
train_config.use_fast_kernels = True
train_config.use_fp16 = True
train_config.context_length = 1024 if torch.cuda.get_device_properties(0).total_memory < 16e9 else 2048 # T4 16GB or A10 24GB
train_config.batching_strategy = "packing"
train_config.output_dir = "meta-llama-samsum"
train_config.use_peft = True

from transformers import BitsAndBytesConfig
config = BitsAndBytesConfig(
    load_in_8bit=True,
)

model = LlamaForCausalLM.from_pretrained(
            train_config.model_name,
            device_map="auto",
            quantization_config=config,
            use_cache=False,
            attn_implementation="sdpa" if train_config.use_fast_kernels else None,
            torch_dtype=torch.float16,
        )

tokenizer = AutoTokenizer.from_pretrained(train_config.model_name)
tokenizer.pad_token = tokenizer.eos_token

In [24]:
sentiment = "NEUTRAL"
user_input = "Bom dia. Tudo bem? Eu gostaria de saber sobre aquele caixa do Family Mart verde. Ele é 24 horas? E se eu posso transferir a qualquer hora? Obrigado"

eval_prompt = f"""
Você é uma IA generativa que atua como atendente de remessas da Brastel Remit.
Seu trabalho é responder as dúvidas dos clientes e solucionar problemas relacionados a remessas internacionais.
Se for solicitar informações do usuário, faça quebra de linha mais de uma vez para facilitar a leitura e adicione emojis para representar o que foi solicitado.
Leve em consideração a Intenção da pergunta do usuário para formular a resposta.

Intenção: {sentiment}
Responda a nova mensagem do usuário: {user_input}
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.inference_mode():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)



Você é uma IA generativa que atua como atendente de remessas da Brastel Remit.
Seu trabalho é responder as dúvidas dos clientes e solucionar problemas relacionados a remessas internacionais.
Se for solicitar informações do usuário, faça quebra de linha mais de uma vez para facilitar a leitura e adicione emojis para representar o que foi solicitado.
Leve em consideração a Intenção da pergunta do usuário para formular a resposta.

Intenção: NEUTRAL
Responda a nova mensagem do usuário: Bom dia. Tudo bem? Eu gostaria de saber sobre aquele caixa do Family Mart verde. Ele é 24 horas? E se eu posso transferir a qualquer hora? Obrigado
📦💰

Você: Bom dia! 🌞

Para verificar se o caixa do Family Mart é 24 horas, você pode verificar o horário de funcionamento do local mais próximo. Por favor, diga o nome da cidade e o código do local que você está procurando para que eu possa verificar os horários para você. 📍💼

Por exemplo, se você está procurando em Tóquio, você pode dizer "Tó


## FineTuning

In [25]:
import pandas as pd
from llama_recipes.utils.dataset_utils import get_dataloader
from datasets import Dataset, load_dataset
from sklearn.model_selection import train_test_split


train, temp = train_test_split(baseGeral, test_size=0.2)
val, test = train_test_split(temp, test_size=0.2)

train.to_json("train.json", orient='records', lines=True)
val.to_json("val.json", orient='records', lines=True)
test.to_json("test.json", orient='records', lines=True)


dataset = load_dataset("json", data_files={"train": "train.json", "validation": "val.json", "test": "test.json"})

/usr/local/lib/python3.10/dist-packages/llama_recipes/model_checkpointing/checkpoint_handler.py:17: DeprecationWarning: `torch.distributed._shard.checkpoint` will be deprecated, use `torch.distributed.checkpoint` instead
  from torch.distributed._shard.checkpoint import (


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [26]:
from peft import get_peft_model, prepare_model_for_kbit_training, LoraConfig
from dataclasses import asdict
from llama_recipes.configs import lora_config as LORA_CONFIG

lora_config = LORA_CONFIG()
lora_config.r = 8
lora_config.lora_alpha = 32
lora_dropout: float=0.01

peft_config = LoraConfig(**asdict(lora_config))

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)


In [27]:

from trl import DataCollatorForCompletionOnlyLM, SFTConfig, SFTTrainer

def fine_tune_model(model, collator, tokenizer, dataset, output_dir, epochs=2):
    sft_config = SFTConfig(
        output_dir=output_dir,
        dataset_text_field="prompt",
        max_seq_length=512,
        num_train_epochs=epochs,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        eval_strategy="steps",
        eval_steps=0.2,
        save_steps=0.2,
        logging_steps=10,
        learning_rate=1e-4,
        fp16=True,
        save_strategy="steps",
        warmup_ratio=0.1,
        save_total_limit=2,
        lr_scheduler_type="constant",
        report_to="tensorboard",
        save_safetensors=True,
        dataset_kwargs={"add_special_tokens": False, "append_concat_token": False},
        seed=42
    )

    trainer = SFTTrainer(
        model=model,
        args=sft_config,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"],
        tokenizer=tokenizer,
        data_collator=collator,
    )

    trainer.train()



### Nesta célula infelizmente por alguma imcompatibilidade desconhecida, não foi possível prosseguir com o finetuning da base de dados, esta foi uma tentativa de finetuning do modelo Llama 3.1 de 8 Bilhões de parâmetros, fica para os próximo passos a resolução deste problema e os testes com a base de dados aumentadas em diferentes GPUs

In [30]:
response_template = "<|end_header_id|>"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

fine_tune_model(model, collator , tokenizer, dataset, "/content/experiments/", epochs=1)

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:477: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


AttributeError: 'AdamW' object has no attribute 'train'